# More Examples

## Logistic Function

In [1]:
import theano
import theano.tensor as T
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))
logistic = theano.function([x], s)
logistic([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

In [2]:
s2 = (1 + T.tanh(x / 2)) / 2
logistic2 = theano.function([x], s2)
logistic2([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

## Computing More than one Thing at the Same Time

In [3]:
a, b = T.dmatrices('a', 'b')
diff = a - b
abs_diff = abs(diff)
diff_squared = diff**2
f = theano.function([a, b], [diff, abs_diff, diff_squared])

In [4]:
f([[1, 1], [1, 1]], [[0, 1], [2, 3]])

[array([[ 1.,  0.],
        [-1., -2.]]), array([[ 1.,  0.],
        [ 1.,  2.]]), array([[ 1.,  0.],
        [ 1.,  4.]])]

## Setting a Default Value for an Argument

In [5]:
from theano import In
from theano import function
x, y = T.dscalars('x', 'y')
z = x + y
f = function([x, In(y, value=1)], z)

In [6]:
f(33)

array(34.0)

In [7]:
f(33, 2)

array(35.0)

In [8]:
x, y, w = T.dscalars('x', 'y', 'w')
z = (x + y) * w
f = function([x, In(y, value=1), In(w, value=2, name='w_by_name')], z)

In [9]:
f(33)

array(68.0)

In [10]:
f(33, 2)

array(70.0)

In [12]:
f(33, 0, 1)

array(33.0)

In [13]:
f(33, w_by_name=1)

array(34.0)

In [14]:
f(33, w_by_name=1, y=0)

array(33.0)

In [15]:
from theano import shared
state = shared(0)
inc = T.iscalar('inc')
accumulator = function([inc], state, updates=[(state, state+inc)])

In [16]:
print(state.get_value())

0


In [17]:
accumulator(1)

array(0)

In [18]:
print(state.get_value())

1


In [19]:
accumulator(300)

array(1)

In [20]:
print(state.get_value())

301


In [21]:
state.get_value(-1)
accumulator(3)

array(301)

In [22]:
print(state.get_value())

304


In [24]:
decrementor = function([inc], state, updates=[(state, state-inc)])
decrementor(2)

array(304)

In [25]:
print(state.get_value())

302


In [26]:
fn_of_state = state * 2 + inc
foo = T.scalar(dtype=state.dtype)
skip_shared = function([inc, foo], fn_of_state, givens=[(state, foo)])
skip_shared(1, 3)

array(7)

In [27]:
print(state.get_value())

302


## Using Random Numbers

### Brief Example

In [28]:
from theano.tensor.shared_randomstreams import RandomStreams
from theano import function
srng = RandomStreams(seed=234)
rv_u = srng.uniform((2,2))
rv_n = srng.normal((2,2))
f = function([], rv_u)
g = function([], rv_n, no_default_updates=True)
nearly_zeros = function([], rv_u + rv_u - 2 * rv_u)

In [29]:
f_val0 = f()
f_val1 = f()

In [30]:
g_val0 = g()
g_val1 = g()

In [31]:
f_val0, f_val1, g_val0, g_val1

(array([[ 0.12672381,  0.97091597],
        [ 0.13989098,  0.88754825]]), array([[ 0.31971415,  0.47584377],
        [ 0.24129163,  0.42046081]]), array([[ 0.37328447, -0.65746672],
        [-0.36302373, -0.97484625]]), array([[ 0.37328447, -0.65746672],
        [-0.36302373, -0.97484625]]))

In [42]:
f(), f(), g(), g()

(array([[ 0.04932455,  0.95363814],
        [ 0.91038487,  0.81128167]]), array([[ 0.07840424,  0.67944917],
        [ 0.33755902,  0.30124475]]), array([[ 0.37328447, -0.65746672],
        [-0.36302373, -0.97484625]]), array([[ 0.37328447, -0.65746672],
        [-0.36302373, -0.97484625]]))

In [37]:
f_val0, f_val1, g_val0, g_val1

(array([[ 0.12672381,  0.97091597],
        [ 0.13989098,  0.88754825]]), array([[ 0.31971415,  0.47584377],
        [ 0.24129163,  0.42046081]]), array([[ 0.37328447, -0.65746672],
        [-0.36302373, -0.97484625]]), array([[ 0.37328447, -0.65746672],
        [-0.36302373, -0.97484625]]))

### Seeding Streams

In [38]:
rng_val = rv_u.rng.get_value(borrow=True)
rng_val.seed(89234)
rv_u.rng.set_value(rng_val, borrow=True)

In [43]:
srng.seed(902340)

In [44]:
### Sharing Streams Between Functions

In [48]:
state_after_v0 = rv_u.rng.get_value().get_state()
nearly_zeros()


array([[ 0.,  0.],
       [ 0.,  0.]])

In [49]:
v1 = f()
rng = rv_u.rng.get_value(borrow=True)
rng.set_state(state_after_v0)
rv_u.rng.set_value(rng, borrow=True)

In [51]:
v2 = f()
v3 = f()

### Copying Random State Between Theano Graphs

In [52]:
from __future__ import print_function
import theano
import numpy
import theano.tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams
from theano.tensor.shared_randomstreams import RandomStreams

In [59]:
class Graph():
    def __init__(self, seed=123):
        self.rng = RandomStreams(seed)
        self.y = self.rng.uniform(size=(1,))

In [60]:
g1 = Graph(seed=123)
f1 = theano.function([], g1.y)

In [61]:
g2 = Graph(seed=987)
f2 = theano.function([], g2.y)

In [62]:
f1()

array([ 0.72803009])

In [63]:
f2()

array([ 0.55056769])

In [64]:
def copy_random_state(g1, g2):
    if isinstance(g1.rng, MRG_RandomStreams):
        g2.rng.rstate = g1.rng.rstate
    for (su1, su2) in zip(g1.rng.state_updates, g2.rng.state_updates):
        su2[0].set_value(su1[0].get_value())

In [65]:
copy_random_state(g1, g2)
f1()

array([ 0.59044123])

In [66]:
f2()

array([ 0.59044123])

### A Real Example: Logistic Regression

In [67]:
import numpy
import theano
import theano.tensor as T
rng = numpy.random

N = 400                                   # training sample size
feats = 784                               # number of input variables

# generate a dataset: D = (input_values, target_class)
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

# Declare Theano symbolic variables
x = T.matrix("x")
y = T.vector("y")

# initialize the weight vector w randomly
#
# this and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)
w = theano.shared(rng.randn(feats), name="w")

# initialize the bias term
b = theano.shared(0., name="b")

print("Initial model:")
print(w.get_value())
print(b.get_value())

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize
gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                          # (we shall return to this in a
                                          # following section of this tutorial)

# Compile
train = theano.function(
          inputs=[x,y],
          outputs=[prediction, xent],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])

print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Final model:
[  2.94099943e-02  -6.54047740e-02  -2.93785903e-02   8.41779181e-02
  -8.91364620e-02  -7.04407636e-02   6.16470293e-02  -4.94700260e-02
   4.84209957e-02   5.14427109e-02   2.09588347e-02   5.28223504e-02
  -3.44340005e-02  -1.95817432e-01  -2.22695853e-01   1.40318546e-01
  -3.37075019e-02   1.34756241e-01  -1.18784573e-01  -7.81133534e-02
   9.97440005e-02  -1.19900925e-01  -5.72592924e-02   5.85934990e-02
   6.03666987e-02  -1.99618537e-01  -6.74732437e-02  -8.22336504e-02
   2.24220439e-03   8.51774724e-02  -5.75302289e-02   1.09690817e-01
  -1.18647703e-01   1.44062717e-01   2.79660234e-02  -3.30743252e-02
  -2.16472526e-01  -6.31222808e-02   4.88357560e-03   1.50647541e-01
  -3.48232975e-02   2.09582596e-02  -1.08252377e-01   4.57015640e-02
   5.27036517e-02   4.78021948e-02  -7.44905570e-02  -1.20122997e-02
   4.32073537e-02  -5.15864063e-02   6.55856644e-03  -8.12355035e-02
   7.72836643e-02   6.01564361e-02  -7.91818601e-02   1.08982483e-01
  -2.08873547e-02   2

Initial model:
[  1.35015741e+00   5.90697404e-01  -4.31974679e-01  -1.11285072e+00
   1.80084284e+00  -1.40169105e+00   2.17947799e+00  -4.29710371e-01
   7.52404144e-01   1.15375594e+00  -1.20908599e+00   5.70505370e-01
   6.78864011e-01  -1.69574838e+00  -8.45369348e-01  -4.89722822e-01
  -1.39895919e+00  -3.20040779e-02   3.65196216e-01  -2.61443594e-02
  -5.30378885e-01   6.10659028e-01   8.00975439e-01   9.38864854e-01
   1.26611255e+00   3.66821769e-01  -4.15414605e-01   5.15755189e-02
   1.27545713e+00   2.19134168e-01  -1.80715842e+00  -1.30955809e+00
   5.90151822e-01   1.10954178e+00  -1.92905906e+00  -6.02278615e-01
  -7.44870586e-01   1.13984065e+00  -1.01973006e+00   3.86553613e-01
   5.97405756e-01   1.21284228e-01  -5.49926318e-01   1.05678082e+00
  -5.50056731e-01  -3.22905857e-01   9.50250386e-02   1.83619329e+00
   4.10831839e-01   3.26949821e-01  -1.61826482e-01  -1.57916172e-01
  -1.48099874e-01  -8.95887118e-01   1.56572239e+00   1.61005233e-02
  -3.77032590e-01  